Notebook Adaptado para Exercício RAG - Fontes Web/PDFs EXPANDIDAS (~110 fontes, Histórico 1990s-Presente).
Usando FAISS (ADA-002) para persistência local. SEM VÍDEOS YOUTUBE. ALTA CARGA DE RECURSOS.


# === 1. Setup (Install and Import) ===

In [166]:
print("Instalando/Atualizando bibliotecas necessárias (incluindo faiss-cpu)...")
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain-openai langchain-core pypdf unstructured requests tiktoken faiss-cpu

import getpass
import os
import requests
import tempfile
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredURLLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.documents import Document
from typing_extensions import List, TypedDict
from langgraph.graph import START, StateGraph
from IPython.display import Image, display
import time
import shutil
import traceback
from urllib.parse import urlparse # To help clean source names

print("Importações concluídas.")

Instalando/Atualizando bibliotecas necessárias (incluindo faiss-cpu)...
Importações concluídas.


In [167]:
# Configurando a chave da API
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    if not openai_api_key: raise ValueError("API Key not found or empty.")
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("Chave da API OpenAI carregada do Colab Secrets.")
except Exception as e:
    print(f"Erro ao buscar chave do Colab Secrets ({e}). Solicitando input.")
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Cole sua Chave da API OpenAI aqui: ")
    if not os.environ["OPENAI_API_KEY"]: print("ERRO: Nenhuma chave da API fornecida. Saindo."); exit()

Chave da API OpenAI carregada do Colab Secrets.


In [168]:
# Inicializando o LLM
LLM_MODEL_NAME = "gpt-4o-mini" # Keep mini for cost during large indexing potentially
try:
    llm = ChatOpenAI(model=LLM_MODEL_NAME, temperature=0, request_timeout=180) # Even longer timeout
    print(f"Modelo LLM ({llm.model_name}) inicializado.")
except Exception as e:
    print(f"Erro crítico ao inicializar o LLM ({LLM_MODEL_NAME}): {e}"); print(traceback.format_exc()[:1000]); exit()

Modelo LLM (gpt-4o-mini) inicializado.


# === 2. Definir Fontes de Dados EXPANDIDAS (~110 Fontes: Blogs/PDFs, ~1990s - Presente) ===

- COMBINAÇÃO DAS LISTAS ANTERIORES + NOVAS, REMOVENDO DUPLICATAS

In [169]:
print("\n--- Definindo Fontes de Dados EXPANDIDAS (~110 Selecionadas: Blogs/PDFs Históricos e Recentes) ---")

# Use sets for easy deduplication
unique_web_urls = set()
unique_pdf_urls = set()

# --- Web Pages / Blog Posts / Articles ---
initial_web_page_urls = [
    # Recentes (2023-2024 foco)
    "https://openai.com/index/hello-gpt-4o/", "https://ai.meta.com/blog/meta-llama-3/",
    "https://blog.google/technology/ai/google-io-2024-ai-announcements/",
    "https://blogs.microsoft.com/ai/introducing-phi-3-redefining-whats-possible-with-slms/",
    "https://huggingface.co/blog/llama3", "https://www.wired.com/story/google-io-2024-biggest-ai-announcements/",
    "https://www.theverge.com/2024/5/13/24155443/openai-gpt-4o-launch-event-live-blog", "https://openai.com/blog/chatgpt/",
    "https://www.technologyreview.com/2024/01/08/1086140/10-breakthrough-technologies-2024/",
    "https://www.wired.com/story/what-is-generative-ai/", "https://www.nature.com/articles/d41586-024-01449-z", # Nature AlphaFold 3

    # Históricos e Conceituais (Pré-2023 + Explainers)
    "http://yann.lecun.com/exdb/lenet/", "https://jalammar.github.io/illustrated-transformer/",
    "https://colah.github.io/posts/2015-08-Understanding-LSTMs/", "https://distill.pub/2016/augmented-rnns/",
    "https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html",
    "https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html",
    "https://openai.com/research/dall-e", "https://deepmind.google/discover/blog/alphago-the-story-so-far/",
    "https://deepmind.google/discover/blog/alphafold-a-solution-to-a-50-year-old-grand-challenge-in-biology/",
    "https://spectrum.ieee.org/deep-learning", "https://www.kdnuggets.com/2017/04/simple-introduction-support-vector-machines.html",
    "https://web.stanford.edu/~hastie/ElemStatLearn/", "https://builtin.com/data-science/reinforcement-learning",
    "https://en.wikipedia.org/wiki/AI_winter", "https://en.wikipedia.org/wiki/Timeline_of_artificial_intelligence",
    "https://ai.stanford.edu/", "https://aiindex.stanford.edu/",

    # Novas adições Web/Blog
    "https://www.nature.com/articles/nature14539", # Deep Learning Review (LeCun, Bengio, Hinton) Nature Page
    "https://blog.google/technology/ai/introducing-gemini-our-largest-and-most-capable-ai-model/", # Gemini Blog
    "https://ai.meta.com/blog/segment-anything-foundation-model-image-segmentation/", # SAM Blog
    "https://pytorch.org/blog/pytorch-2-release/", # PyTorch 2.0 Blog
    "https://timdettmers.com/2023/01/30/vicuna-the-missing-link-between-llm-and-chatgpt/", # Vicuna Blog Post (Example)
    "https://lilianweng.github.io/posts/2023-06-23-agent/", # Lilian Weng AI Agents
    "https://lilianweng.github.io/posts/2021-07-11-diffusion-models/", # Lilian Weng Diffusion Models
    "https://sebastianraschka.com/blog/2023/llm-finetuning-explained.html", # Raschka Finetuning Explainer
    "https://huggingface.co/blog/stable_diffusion", # HF Stable Diffusion Blog
    "https://www.deeplearning.ai/the-batch/", # The Batch Newsletter (Link to archive/main page)
    "https://developer.nvidia.com/blog/tag/generative-ai/", # Nvidia Generative AI Tag
    "https://research.facebook.com/blog/2017/01/fair-open-sources-fasttext/", # Facebook fastText Blog
    "https://www.explainxkcd.com/wiki/index.php/2860:_Unintended_Uses", # XKCD on Alignment (Context)
    "https://futureoflife.org/ai-safety-research/", # FLI AI Safety Page
    "https://www.eff.org/ai/resources", # EFF AI Resources Page
    "https://en.wikipedia.org/wiki/Convolutional_neural_network", # Wikipedia CNN
    "https://en.wikipedia.org/wiki/Recurrent_neural_network", # Wikipedia RNN
    "https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53", # TDS CNN
    "https://towardsdatascience.com/introduction-to-reinforcement-learning-rl-part-1-e1d95a6c96ac", # TDS RL Intro
    "https://cset.georgetown.edu/publication/understanding-ai-ethics-and-safety-a-guide-for-the-conscientious-objector/", # CSET Ethics Example
    "https://aws.amazon.com/what-is/vector-database/", # AWS Vector DB Explainer
    "https://openai.com/policies/usage-policies", # OpenAI Usage Policies
    "https://www.assemblyai.com/blog/diffusion-models-for-visual-data-a-deep-dive/", # AssemblyAI Diffusion Blog
    "https://magazine.towardsdatascience.com/attention-is-all-you-need-the-neural-network-that-revolutionised-nlp-60ab88d3e85f", # TDS Transformer/Attention
    "https://developer.ibm.com/articles/intro-computational-linguistics/", # IBM Computational Linguistics Intro
    "https://huyenchip.com/blog/", # Chip Huyen's Blog (Link main page)
    "https://blog.research.google/", # Google Research Blog (Main)
    "https://github.blog/category/copilot/", # GitHub Copilot Blog Category
    "https://aws.amazon.com/blogs/machine-learning/", # AWS ML Blog (Main)
]

# --- PDFs (arXiv / Reports) ---
initial_pdf_urls = [
    # Recentes (2023-2024 Foco)
    "https://arxiv.org/pdf/2404.11225", "https://arxiv.org/pdf/2404.14219",
    "https://cdn.openai.com/gpt-4o-system-card.pdf", "https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf",
    "https://arxiv.org/pdf/2305.18290.pdf", "https://arxiv.org/pdf/2303.10130.pdf",

    # Históricos (Pré-2023 - Deep Learning Era etc.)
    "https://arxiv.org/pdf/1301.3781.pdf", "https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf", # AlexNet
    "https://arxiv.org/pdf/1706.03762.pdf", "https://arxiv.org/pdf/1810.04805.pdf", # BERT
    "https://arxiv.org/pdf/1406.2661.pdf", "https://arxiv.org/pdf/2005.14165.pdf", # GPT-3
    "https://arxiv.org/pdf/1409.3215.pdf", "https://arxiv.org/pdf/1409.0473.pdf", # Bahdanau Attention
    "https://arxiv.org/pdf/2103.00020.pdf", "https://arxiv.org/pdf/2006.11239.pdf", # DDPM
    "https://arxiv.org/pdf/2203.02155.pdf", "https://arxiv.org/pdf/2205.11487.pdf", # Chain-of-Thought
    "https://arxiv.org/pdf/1511.06434.pdf", "https://arxiv.org/pdf/1506.02640.pdf", # Faster R-CNN
    "https://arxiv.org/pdf/1312.6114.pdf", "https://arxiv.org/pdf/1603.05027.pdf", # WaveNet
    "https://arxiv.org/pdf/1703.06870.pdf", # SAGAN

    # Novas adições PDF
    "https://arxiv.org/pdf/1512.03385.pdf", # ResNet
    "https://arxiv.org/pdf/1412.6980.pdf", # Adam Optimizer
    "https://arxiv.org/pdf/1502.01852.pdf", # Batch Normalization
    "https://arxiv.org/pdf/1710.10903.pdf", # StyleGAN
    "https://arxiv.org/pdf/1911.11427.pdf", # BART
    "https://arxiv.org/pdf/1907.11692.pdf", # RoBERTa
    "https://arxiv.org/pdf/2010.11929.pdf", # Vision Transformer (ViT)
    "https://arxiv.org/pdf/2106.09685.pdf", # DreamerV2 (RL)
    "https://arxiv.org/pdf/2104.09755.pdf", # FLAN
    "https://arxiv.org/pdf/2112.10752.pdf", # GLIDE (Diffusion)
    "https://arxiv.org/pdf/2204.02311.pdf", # LaMDA
    # RLHF Survey needed better link, potentially find openreview or blog summary
    "https://arxiv.org/pdf/2304.08485.pdf", # Self-Instruct
    "https://arxiv.org/pdf/2302.13971.pdf", # LLaMA (Original)
    "https://arxiv.org/pdf/2306.15595.pdf", # Segment Anything (SAM)
    "https://arxiv.org/pdf/2305.11206.pdf", # Voyager (Agents)
    "https://arxiv.org/pdf/2308.10383.pdf", # Llama 2
    "https://arxiv.org/pdf/2307.09288.pdf", # Toolformer
    "https://arxiv.org/pdf/2401.04088.pdf", # Mixtral of Experts
    "https://arxiv.org/pdf/2402.17764.pdf", # Gemma Paper
    "https://arxiv.org/pdf/2304.07193.pdf", # AlphaDev
    "https://arxiv.org/pdf/2310.13592.pdf", # Zephyr
    "https://arxiv.org/pdf/cs/9704103.pdf", # Boosting (Schapire) - Check accessibility
    "https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf", # TensorFlow Whitepaper
    "https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf", # Glorot Initialization
    "https://arxiv.org/pdf/1707.06347.pdf", # PPO (RL)
    "https://arxiv.org/pdf/1909.11553.pdf", # T5
    "https://arxiv.org/pdf/2311.10122.pdf", # GNN Survey
    "https://arxiv.org/pdf/2306.06174.pdf", # Formal Algorithms for Transformers
    # Find AI Index 2023/2024 PDFs if possible, e.g. searching "Stanford HAI AI Index 2024 PDF"
]


--- Definindo Fontes de Dados EXPANDIDAS (~110 Selecionadas: Blogs/PDFs Históricos e Recentes) ---


In [170]:
# Deduplicate and finalize lists
web_page_urls = sorted(list(set(initial_web_page_urls)))
pdf_urls = sorted(list(set(initial_pdf_urls)))

final_total_sources = len(web_page_urls) + len(pdf_urls)
print(f"\nFinal Unique Web Sources Defined: {len(web_page_urls)}")
print(f"Final Unique PDF Sources Defined: {len(pdf_urls)}")
print(f"FINAL GRAND TOTAL Unique Sources: {final_total_sources}")
print("------------------------------------")


Final Unique Web Sources Defined: 57
Final Unique PDF Sources Defined: 51
FINAL GRAND TOTAL Unique Sources: 108
------------------------------------


# === 3. Carregar Dados ===

In [171]:
print(f"\nIniciando carregamento para {final_total_sources} fontes (Web/PDF) - ISSO PODE DEMORAR MUITO...")
# ...(Loading logic using the combined, deduplicated lists remains conceptually the same)...
# ...(Includes error handling, temp file logic for PDFs)...
all_docs = []
loading_errors = []


Iniciando carregamento para 108 fontes (Web/PDF) - ISSO PODE DEMORAR MUITO...


In [172]:
# --- Helper Function ---
def load_with_error_handling(loader_func, source_identifier):
    try:
        docs = loader_func()
        if not isinstance(docs, list): docs = list(docs)
        if docs: print(f"   + OK ({len(docs)}): {source_identifier}"); all_docs.extend(docs)
        else: print(f"   - WARN (Empty): {source_identifier}")
        time.sleep(0.3) # Shorter sleep to speed up slightly
        return True
    except Exception as e:
        error_msg = f"ERROR loading '{source_identifier}': {str(e)[:200]}..." # Shorter error log
        print(f"   ! {error_msg}")
        loading_errors.append(error_msg)
        time.sleep(0.3)
        return False

In [173]:
# 3.1 Load Web Pages
if web_page_urls:
    print(f"\n-- Loading {len(web_page_urls)} Web Pages...")
    web_start_time = time.time()
    web_loader = UnstructuredURLLoader(urls=web_page_urls, ssl_verify=False,
                                       headers={'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'},
                                       mode="elements", continue_on_failure=True, show_progress_bar=True)
    load_with_error_handling(web_loader.load, "Web Batch")
    print(f"-- Web loading finished (Time: {time.time() - web_start_time:.2f}s)")


-- Loading 57 Web Pages...


  2%|▏         | 1/57 [00:00<00:10,  5.27it/s]/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.googleblog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'blog.research.google'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'research.google'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  4%|▎        

   + OK (4597): Web Batch
-- Web loading finished (Time: 29.31s)


In [174]:
# 3.2 Load PDFs
if pdf_urls:
    print(f"\n-- Loading {len(pdf_urls)} PDFs...")
    pdf_start_time = time.time()
    for i, url in enumerate(pdf_urls):
        temp_pdf_path = None
        print(f"   ({i+1}/{len(pdf_urls)}) PDF: {os.path.basename(urlparse(url).path)}...") # Show progress
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            response = requests.get(url, stream=True, headers=headers, timeout=120)
            response.raise_for_status()
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
                temp_pdf_path = temp_file.name
                for chunk in response.iter_content(chunk_size=1024*1024): temp_file.write(chunk) # 1MB chunks download
            pdf_loader_instance = PyPDFLoader(temp_pdf_path, extract_images=False)
            def load_pdf_wrapper():
                loaded_pdf_docs = pdf_loader_instance.load() # Use basic load, splitting later
                for doc in loaded_pdf_docs: doc.metadata['source'] = url
                return loaded_pdf_docs
            load_with_error_handling(load_pdf_wrapper, url)
        except Exception as e_pdf: # Catch all exceptions during PDF handling for this URL
            error_msg = f"ERROR for PDF '{url}': {str(e_pdf)[:200]}..."
            print(f"   ! {error_msg}")
            loading_errors.append(error_msg)
        finally:
            if temp_pdf_path and os.path.exists(temp_pdf_path):
                try: os.remove(temp_pdf_path)
                except OSError as e_rm: print(f"   ! Warning: Failed removing {temp_pdf_path}: {e_rm}")
            time.sleep(0.5) # Slightly longer sleep between PDFs
    print(f"-- PDF loading finished (Time: {time.time() - pdf_start_time:.2f}s).")


print(f"\nTotal documents loaded before splitting: {len(all_docs)}")
if loading_errors:
    print(f"\nWARNING: Encountered {len(loading_errors)} errors during loading. Knowledge base might be incomplete.")
    # Optionally print unique errors here if needed


-- Loading 51 PDFs...
   (1/51) PDF: 1301.3781.pdf...
   + OK (12): https://arxiv.org/pdf/1301.3781.pdf
   (2/51) PDF: 1312.6114.pdf...
   + OK (14): https://arxiv.org/pdf/1312.6114.pdf
   (3/51) PDF: 1406.2661.pdf...
   + OK (9): https://arxiv.org/pdf/1406.2661.pdf
   (4/51) PDF: 1409.0473.pdf...
   + OK (15): https://arxiv.org/pdf/1409.0473.pdf
   (5/51) PDF: 1409.3215.pdf...
   + OK (9): https://arxiv.org/pdf/1409.3215.pdf
   (6/51) PDF: 1412.6980.pdf...
   + OK (15): https://arxiv.org/pdf/1412.6980.pdf
   (7/51) PDF: 1502.01852.pdf...
   + OK (11): https://arxiv.org/pdf/1502.01852.pdf
   (8/51) PDF: 1506.02640.pdf...
   + OK (10): https://arxiv.org/pdf/1506.02640.pdf
   (9/51) PDF: 1511.06434.pdf...
   + OK (16): https://arxiv.org/pdf/1511.06434.pdf
   (10/51) PDF: 1512.03385.pdf...
   + OK (12): https://arxiv.org/pdf/1512.03385.pdf
   (11/51) PDF: 1603.05027.pdf...
   + OK (15): https://arxiv.org/pdf/1603.05027.pdf
   (12/51) PDF: 1703.06870.pdf...
   + OK (12): https://arxiv.org

   + OK (8): https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
   (49/51) PDF: c399862d3b9d6b76c8436e924a68c45b-Paper.pdf...
   + OK (9): https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf
   (50/51) PDF: 45530.pdf...


   + OK (8): https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf
   (51/51) PDF: gemma-report.pdf...
   + OK (17): https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf
-- PDF loading finished (Time: 109.35s).

Total documents loaded before splitting: 5770



# === 4. Processar Dados (Split) ===
 - ...(Splitting logic remains similar, check params)

In [175]:
#...(Splitting logic remains similar, check params)
all_splits = []
if all_docs:
    print(f"\nSplitting {len(all_docs)} documents into chunks...")
    # Larger overlap might help with context across ~1k chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=250, add_start_index=True,
        separators=["\n\n", "\n", ". ", "? ", "! ", ";", ",", " ", ""], keep_separator=False
    )
    try:
      start_split_time = time.time()
      all_splits = text_splitter.split_documents(all_docs)
      split_duration = time.time() - start_split_time
      print(f"Splitting complete: {len(all_splits)} chunks created (Time: {split_duration:.2f}s).")
      if not all_splits and all_docs: print("WARNING: Splitting yielded 0 chunks.")
    except Exception as split_e: print(f"CRITICAL Error during splitting: {split_e}"); all_splits = []
else: print("No documents loaded to split.")


Splitting 5770 documents into chunks...
Splitting complete: 9687 chunks created (Time: 0.79s).


# === 5. Embeddings & Vector Store ===

In [176]:
vector_store = None
# !! NOVO NOME DE ÍNDICE PARA O DATASET EXPANDIDO SEM YT!!
FAISS_INDEX_PATH = "faiss_index_ai_history_expanded_ada_v1"
# !! TRUE para a PRIMEIRA execução desta versão!!
FORCE_RECREATE_INDEX = True
EMBEDDING_MODEL_NAME = "text-embedding-ada-002" # Still using ada-002

if all_splits:
    print(f"\nConfiguring FAISS (Path: '{FAISS_INDEX_PATH}', Recreate: {FORCE_RECREATE_INDEX})...")
    try:
        embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
        print(f"Using embedding model: {embeddings.model}")

        if os.path.exists(FAISS_INDEX_PATH) and not FORCE_RECREATE_INDEX:
            print(f"-- Loading existing FAISS index for {embeddings.model} from '{FAISS_INDEX_PATH}'...")
            # Load logic... (as before)
            vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
            print("-- Index loaded.")
        else:
             if os.path.exists(FAISS_INDEX_PATH):
                 print(f"-- Removing old/incompatible FAISS index: '{FAISS_INDEX_PATH}'...")
                 try: shutil.rmtree(FAISS_INDEX_PATH); print("-- Old index removed.")
                 except OSError as e_rm: print(f"-- Warning: Failed removing old index: {e_rm}.")

             # Create - THIS WILL TAKE A LONG TIME AND USE SIGNIFICANT API CREDITS
             print(f"-- Creating NEW FAISS index from {len(all_splits)} chunks using {embeddings.model}...")
             print("-- This involves sending data to OpenAI for embeddings and can take time/cost credits --")
             start_create_time = time.time()
             vector_store = FAISS.from_documents(all_splits, embeddings)
             create_duration = time.time() - start_create_time
             print(f"-- Index created (Time: {create_duration:.2f}s).")

             # Save
             print(f"-- Saving FAISS index to '{FAISS_INDEX_PATH}'...")
             #...(Save logic as before)
             if not os.path.exists(FAISS_INDEX_PATH): os.makedirs(FAISS_INDEX_PATH)
             vector_store.save_local(FAISS_INDEX_PATH)
             print("-- Index saved.")

    except Exception as e_faiss: print(f"CRITICAL FAISS Error ({embeddings.model}): {e_faiss}"); print(traceback.format_exc()[:1000]); vector_store = None
else: print("No chunks to process. FAISS store not configured.")


Configuring FAISS (Path: 'faiss_index_ai_history_expanded_ada_v1', Recreate: True)...
Using embedding model: text-embedding-ada-002
-- Removing old/incompatible FAISS index: 'faiss_index_ai_history_expanded_ada_v1'...
-- Old index removed.
-- Creating NEW FAISS index from 9687 chunks using text-embedding-ada-002...
-- This involves sending data to OpenAI for embeddings and can take time/cost credits --
-- Index created (Time: 41.67s).
-- Saving FAISS index to 'faiss_index_ai_history_expanded_ada_v1'...
-- Index saved.


# === 6. Definir a Cadeia RAG (LangGraph) ===
- ...(RAG Graph setup mostly the same, but update prompt and consider retriever K)...

In [177]:
print("\nDefining RAG graph...")
prompt = None; rag_graph = None

if vector_store and llm:
    try:
        prompt = hub.pull("rlm/rag-prompt") # Base prompt
        # UPDATED PROMPT FOR EXPANDED SCOPE
        prompt.messages[0].prompt.template = """Aja com um assistente de IA especialista em história da Inteligência Artificial.
        Você tem profundo conhecimento dos principais avanços, marcos, paradigmas e transformações ocorridas na IA desde 1990 até o presente.
        Seu treinamento é baseado exclusivamente em artigos da web e documentos PDF, especialmente aquele fornecido nesta conversa.
        Seu objetivo é responder a perguntas específicas sobre a história e os avanços da IA de forma factual, clara e concisa.
        Você deve usar estritamente o conteúdo do(s) documento(s) fornecido(s) como referência para suas respostas.
        Instruções passo a passo:
        Leia a pergunta feita pelo usuário.
        Reflita se há uma resposta direta, clara e verificável dentro do(s) documento(s) fornecido(s).
        Se houver uma resposta:
        Escreva de forma clara e concisa (em 4 a 6 frases no máximo).
        Separe os parágrafos para facilitar a leitura.
        Limite-se ao contexto fornecido.
        Se não houver resposta ou evidência suficiente no conteúdo fornecido:
        Responda exatamente com: "Com base nas informações disponíveis, não sei responder a essa pergunta."
        Nunca utilize conhecimento externo, mesmo que você saiba a resposta.
        Responda em Português.

Contexto Fornecido:
{context}

Pergunta do Usuário: {question}

Resposta Factual e Concisa:"""
        print("RAG prompt adjusted for WIDE historical scope & Portuguese.")

        class State(TypedDict): question: str; context: List[Document]; answer: str

        def retrieve(state: State):
            question = state["question"]
            # INCREASE K further due to very large index? Test this value.
            k_results = 10 # Increased k
            relevance_threshold = 0.70 # Keep threshold, maybe adjust later
            print(f"-- Retrieving context (k={k_results}, thr={relevance_threshold}) via FAISS for: '{question}' --")
            try:
                 docs_scores = vector_store.similarity_search_with_relevance_scores(question, k=k_results)
                 filtered_docs = [doc for doc, score in docs_scores if score >= relevance_threshold]
                 print(f"-- Retrieved {len(docs_scores)} chunks, {len(filtered_docs)} relevant.")
                 if not filtered_docs and docs_scores: # Fallback
                     filtered_docs = [doc for doc, score in docs_scores[:min(3, len(docs_scores))]] # Keep top 3 if none pass threshold
                     print(f"-- Threshold not met, using top {len(filtered_docs)} chunks instead.")
                 return {"context": filtered_docs, "question": question}
            except Exception as retrieve_e: print(f"! FAISS Search Error: {retrieve_e}"); return {"context": [], "question": question}

        def generate(state: State):
             #...(Generate logic mostly same, ensure formatting works)...
             print("-- Generating answer based on context --")
             question = state["question"]; context = state["context"]
             if not context: return {"answer": "Com base nas informações disponíveis, não sei responder a essa pergunta."}
             docs_content_list = []
             for i, doc in enumerate(context):
                  source=doc.metadata.get('source','?'); page=doc.metadata.get('page','');
                  s_display = f"Fonte {i+1}: {os.path.basename(urlparse(source).path)}" # Shorter display name
                  if page!='': s_display+=f" (P.{page+1})"
                  docs_content_list.append(f"{s_display}\n{doc.page_content}")
             docs_content = "\n\n---\n".join(docs_content_list)
             try:
                 messages = prompt.invoke({"question": question, "context": docs_content})
                 response = llm.invoke(messages); print("-- LLM response received --")
                 return {"answer": response.content.strip()}
             except Exception as invoke_e: print(f"! LLM Error: {invoke_e}"); return {"answer": f"Error generating response: {type(invoke_e).__name__}"}

        # Build Graph (same structure)
        graph_builder = StateGraph(State); graph_builder.add_node("retrieve", retrieve); graph_builder.add_node("generate", generate)
        graph_builder.set_entry_point("retrieve"); graph_builder.add_edge("retrieve", "generate"); graph_builder.set_finish_point("generate")
        rag_graph = graph_builder.compile()
        print("RAG graph compiled successfully.")

    except Exception as graph_e: print(f"CRITICAL Error building RAG graph: {graph_e}"); print(traceback.format_exc()[:1000]); rag_graph = None
else: print("Could not define RAG graph (Missing LLM or Vector Store).")


Defining RAG graph...
RAG prompt adjusted for WIDE historical scope & Portuguese.
RAG graph compiled successfully.


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# === 7. Consultar o Assistente ===

In [178]:
if rag_graph:
    print(f"\n=== RAG Assistant (EXPANDED Web/PDFs, History 1990s-Present, Embed: {EMBEDDING_MODEL_NAME}) Ready ===")
    #...(Query loop remains the same)...
    print(f"Using FAISS index from: '{FAISS_INDEX_PATH}'")
    print("Ask questions about AI history/concepts/models. Type 'sair' to exit.")
    while True:
        try:
            user_question = input("\nYour question: ")
            if user_question.lower() == 'sair': break
            if not user_question.strip(): continue
            start_query_time = time.time()
            final_state = rag_graph.invoke({"question": user_question})
            query_duration = time.time() - start_query_time
            print(f"\nAssistant's Answer: (Replied in {query_duration:.2f}s)")
            print(final_state.get('answer', "Error: No answer found."))
        except KeyboardInterrupt: print("\nExiting..."); break
        except Exception as loop_e: print(f"\nUnexpected error in query loop: {loop_e}")
    print("\nAssistant terminated.")
else:
    print("\nRAG assistant could not be started due to critical setup errors.")


=== RAG Assistant (EXPANDED Web/PDFs, History 1990s-Present, Embed: text-embedding-ada-002) Ready ===
Using FAISS index from: 'faiss_index_ai_history_expanded_ada_v1'
Ask questions about AI history/concepts/models. Type 'sair' to exit.

Your question: Compare as abordagens de pré-treinamento do BERT e do GPT original.
-- Retrieving context (k=10, thr=0.7) via FAISS for: 'Compare as abordagens de pré-treinamento do BERT e do GPT original.' --
-- Retrieved 10 chunks, 10 relevant.
-- Generating answer based on context --
-- LLM response received --

Assistant's Answer: (Replied in 6.09s)
As abordagens de pré-treinamento do BERT e do GPT original diferem significativamente em sua arquitetura e metodologia. O GPT utiliza um modelo de Transformer unidirecional, treinando de forma sequencial da esquerda para a direita, enquanto o BERT adota um modelo de Transformer bidirecional, permitindo que as representações sejam condicionadas tanto ao contexto à esquerda quanto à direita em todas as cam